In [27]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from functools import reduce

In [28]:
# Import data
with open('data/disaster_messages.csv', encoding='utf-8') as f:
    df_messages = pd.read_csv(f)

print(df_messages.shape)
print(df_messages.head())

with open('data/disaster_categories.csv', encoding='utf-8') as f:
    df_categories = pd.read_csv(f)

print(df_categories.shape)
print(df_categories.head())

(26248, 4)
   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct  
(26248, 2)
   id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-0;medi...
3   9  rela

In [29]:
column_names = list(map(lambda s: s[:-2], df_categories['categories'].str.split(';')[0]))
df_dummies = df_categories['categories'].str.split(';', expand=True).apply(lambda x: x.str[-1] == '1')
df_dummies.columns = column_names
df_merged = pd.concat([df_categories, df_dummies], axis=1).drop(columns=['categories', 'id'])
df_merged.melt()[df_merged.melt()["value"] == True]

,variable,value
0,related,True
1,related,True
2,related,True
3,related,True
4,related,True
...,...,...
944843,direct_report,True
944893,direct_report,True
944909,direct_report,True
944910,direct_report,True


In [30]:
df_merged = pd.concat([df_messages, df_categories], axis=1)
print(df_merged.shape)
print(df_merged.head())

(26248, 6)
   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  id  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct   2   
1                 Cyclone nan fini osinon li pa fini  direct   7   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct   8   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct   9   
4  facade ouest d Haiti et le reste du pays aujou...  direct  12   

                                          categories  
0  related-1;request-0;offer-0;aid_related-0;medi...  
1  related-1;request-0;offer-0;aid_related-1;medi...  
2  related-1;request-0;offer-0;aid_related-0;medi...  

In [31]:
df_merged['genre'].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [32]:
# Transform features
drop_features = lambda df: df.drop(columns=['id', 'original'])

def get_categories(df):
    column_names = list(map(lambda s: s[:-2], df['categories'].str.split(';')[0]))
    df_dummies = df['categories'].str.split(';', expand=True).apply(lambda x: x.str[-1] == '1')
    df_dummies.columns = column_names
    df_merged = pd.concat([df, df_dummies], axis=1).drop(columns='categories')
    return df_merged

drop_duplicates = lambda df: df.drop_duplicates()

fns = [drop_features, get_categories, drop_duplicates]
df_trans = reduce(lambda res, fn: fn(res), fns, df_merged)
print(df_trans.shape)
print(df_trans.head())

(26215, 38)
                                             message   genre  related  \
0  Weather update - a cold front from Cuba that c...  direct     True   
1            Is the Hurricane over or is it not over  direct     True   
2                    Looking for someone but no name  direct     True   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct     True   
4  says: west side of Haiti, rest of the country ...  direct     True   

   request  offer  aid_related  medical_help  medical_products  \
0    False  False        False         False             False   
1    False  False         True         False             False   
2    False  False        False         False             False   
3     True  False         True         False              True   
4    False  False        False         False             False   

   search_and_rescue  security  ...  aid_centers  other_infrastructure  \
0              False     False  ...        False                 False   
1   

In [33]:
df_trans["security"].sum()

471

In [35]:
# Save to an SQLite database
engine = create_engine('sqlite:///data/DisasterResponse.db')

with engine.connect() as connection:
    df_trans.to_sql('messages', connection, index=False, if_exists='replace')
    result = connection.execute("SELECT * FROM Messages LIMIT 5")
    print(result.fetchall())

[('Weather update - a cold front from Cuba that could pass over Haiti', 'direct', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), ('Is the Hurricane over or is it not over', 'direct', 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0), ('Looking for someone but no name', 'direct', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), ('UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.', 'direct', 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), ('says: west side of Haiti, rest of the country today and tonight', 'direct', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]
